## fine tuning using data from retail

In [1]:
import pandas as pd
import torch
from torch.utils.data import DataLoader
from sentence_transformers import losses, util
from sentence_transformers import LoggingHandler, SentenceTransformer, evaluation
from sentence_transformers.readers import InputExample
import logging
from datetime import datetime
import csv
import os
from zipfile import ZipFile
import random

In [2]:
pd.set_option('display.max_colwidth',None)

In [3]:
#### Just some code to print debug information to stdout
logging.basicConfig(format='%(asctime)s - %(message)s',
                    datefmt='%Y-%m-%d %H:%M:%S',
                    level=logging.INFO,
                    handlers=[LoggingHandler()])
logger = logging.getLogger(__name__)
#### /print debug information to stdout

In [4]:
torch_device = 'cuda' if torch.cuda.is_available() else 'cpu'
# torch_device = 'cpu'

model = SentenceTransformer('multi-qa-MiniLM-L6-cos-v1',
                           device=torch_device)
num_epochs = 1#10
train_batch_size = 16

2022-11-10 12:22:07 - Load pretrained SentenceTransformer: multi-qa-MiniLM-L6-cos-v1


In [5]:
#As distance metric, we use cosine distance (cosine_distance = 1-cosine_similarity)
distance_metric = losses.SiameseDistanceMetric.COSINE_DISTANCE

#Negative pairs should have a distance of at least 0.5
margin = 0.5

dataset_path = os.path.join(os.path.dirname(os.path.dirname(os.getcwd())), 
                            'data/retail_data/')

model_save_path = os.path.join(os.path.dirname(os.path.dirname(os.getcwd())), 
                               'data/output/training_retailConstrativeLoss-')\
+datetime.now().strftime("%Y-%m-%d_%H-%M-%S")


os.makedirs(model_save_path, exist_ok=True)

In [6]:
if not os.path.exists(dataset_path):
    util.http_get(url='https://pages.cs.wisc.edu/~anhai/data1/deepmatcher_data/Structured/Amazon-Google/exp_data/tableA.csv',
                  path=os.path.join(dataset_path,'tableA.csv'))
    util.http_get(url='https://pages.cs.wisc.edu/~anhai/data1/deepmatcher_data/Structured/Amazon-Google/exp_data/tableB.csv',
                  path=os.path.join(dataset_path,'tableB.csv'))
    util.http_get(url='https://pages.cs.wisc.edu/~anhai/data1/deepmatcher_data/Structured/Amazon-Google/exp_data/test.csv',
                  path=os.path.join(dataset_path,'test.csv'))
    util.http_get(url='https://pages.cs.wisc.edu/~anhai/data1/deepmatcher_data/Structured/Amazon-Google/exp_data/train.csv',
                  path=os.path.join(dataset_path,'train.csv'))
    util.http_get(url='https://pages.cs.wisc.edu/~anhai/data1/deepmatcher_data/Structured/Amazon-Google/exp_data/valid.csv',
                  path=os.path.join(dataset_path,'valid.csv'))

In [53]:
# Getting de data for the correct format
tableA = pd.read_csv(os.path.join(dataset_path,'tableA.csv'))
tableA['id'] = 'a_' + tableA.id.astype(str).str.zfill(4)
tableB = pd.read_csv(os.path.join(dataset_path,'tableB.csv'))
tableB['id'] = 'b_' + tableB.id.astype(str).str.zfill(4)
train = pd.read_csv(os.path.join(dataset_path,'train.csv'))
train['ltable_id'] = 'a_' + train.ltable_id.astype(str).str.zfill(4)
train['rtable_id'] = 'b_' + train.rtable_id.astype(str).str.zfill(4)
test = pd.read_csv(os.path.join(dataset_path,'test.csv'))
test['ltable_id'] = 'a_' + test.ltable_id.astype(str).str.zfill(4)
test['rtable_id'] = 'b_' + test.rtable_id.astype(str).str.zfill(4)
valid = pd.read_csv(os.path.join(dataset_path,'valid.csv'))
valid['ltable_id'] = 'a_' + valid.ltable_id.astype(str).str.zfill(4)
valid['rtable_id'] = 'b_' + valid.rtable_id.astype(str).str.zfill(4)

In [54]:
valid

,ltable_id,rtable_id,label
0,a_0897,b_0075,0
1,a_0827,b_2333,0
2,a_0600,b_2176,0
3,a_1284,b_2765,0
4,a_0396,b_3087,0
...,...,...,...
2288,a_1016,b_3189,0
2289,a_0661,b_1213,0
2290,a_0654,b_0448,0
2291,a_0498,b_2547,1


0       b_0000
1       b_0001
2       b_0002
3       b_0003
4       b_0004
         ...  
3221    b_3221
3222    b_3222
3223    b_3223
3224    b_3224
3225    b_3225
Name: id, Length: 3226, dtype: object

In [8]:
train_pairs = (train#[train.label == 1]
               .merge(tableA, left_on='ltable_id', right_on='id')
               .merge(tableB, left_on='rtable_id', right_on='id')
              )[['title_x', 'title_y', 'label']]

In [9]:
######### Read train data  ##########
# Read train data
train_samples = []
for index, row in train_pairs.iterrows():
    sample = InputExample(texts=[row['title_x'], row['title_y']], label=int(row['label']))
    train_samples.append(sample)

In [10]:
train_dataloader = DataLoader(train_samples, shuffle=True, batch_size=train_batch_size)
train_loss = losses.OnlineContrastiveLoss(model=model, distance_metric=distance_metric, margin=margin)

In [16]:
valid_pairs = (valid#[valid.label == 1]
               .merge(tableA, left_on='ltable_id', right_on='id')
               .merge(tableB, left_on='rtable_id', right_on='id')
              )[['title_x', 'title_y', 'label']]

In [21]:
################### Development  Evaluators ##################
# We add 3 evaluators, that evaluate the model on Duplicate Questions pair classification,
# Duplicate Questions Mining, and Duplicate Questions Information Retrieval
evaluators = []

###### Classification ######
# Given (quesiton1, question2), is this a duplicate or not?
# The evaluator will compute the embeddings for both questions and then compute
# a cosine similarity. If the similarity is above a threshold, we have a duplicate.
dev_sentences1 = []
dev_sentences2 = []
dev_labels = []

for index, row in valid_pairs.iterrows():
    dev_sentences1.append(row['title_x'])
    dev_sentences2.append(row['title_y'])
    dev_labels.append(int(row['label']))


binary_acc_evaluator = evaluation.BinaryClassificationEvaluator(dev_sentences1, 
                                                                dev_sentences2, 
                                                                dev_labels)
evaluators.append(binary_acc_evaluator)

In [34]:
max_dev_samples = 10
dev_sentences = {}
for index, row in tableB.iterrows():
    dev_sentences[row['id']] = row['title']
    
    if len(dev_sentences) >= max_dev_samples:
        break    

dev_sentences

{0: 'learning quickbooks 2007',
 1: 'superstart ! fun with reading & writing !',
 2: 'qb pos 6.0 basic software',
 3: 'math missions : the amazing arcade adventure ( grades 3-5 )',
 4: 'production prem cs3 mac upgrad',
 5: 'video studio 11 plus',
 6: 'edius pro 4',
 7: 'qb pos 6.0 pro multi store sw',
 8: 'qb credit card kit 3.0',
 9: 'vegas movie studiodvd +8 plat'}

In [29]:
tableA

,id,title,manufacturer,price
0,0,clickart 950 000 premier image pack ( dvd-rom ),broderbund,NaN
1,1,ca international arcserve lap/desktop oem 30pk,computer associates,NaN
2,2,noah 's ark activity center ( jewel case ages 3-8 ),victory multimedia,NaN
3,3,peachtree by sage premium accounting for nonprofits 2007,sage software,599.99
4,4,singing coach unlimited,carry-a-tune technologies,99.99
...,...,...,...,...
1358,1358,flash remoting 1 alp ret eng cd 2u,adobe,3314.09
1359,1359,shapes,school zone,9.99
1360,1360,dragon naturally speaking standard v9,nuance-communications-inc .,99.99
1361,1361,mediarecover,aladdin systems,29.99


In [ ]:
###### Duplicate Questions Mining ######
# Given a large corpus of questions, identify all duplicates in that corpus.

# For faster processing, we limit the development corpus to only 10,000 sentences.
# max_dev_samples = 10000
max_dev_samples = 1000
dev_sentences = {}
dev_duplicates = []
with open(os.path.join(dataset_path, "duplicate-mining/dev_corpus.tsv"), encoding='utf8') as fIn:
    reader = csv.DictReader(fIn, delimiter='\t', quoting=csv.QUOTE_NONE)
    for row in reader:
        dev_sentences[row['qid']] = row['question']

        if len(dev_sentences) >= max_dev_samples:
            break

with open(os.path.join(dataset_path, "duplicate-mining/dev_duplicates.tsv"), encoding='utf8') as fIn:
    reader = csv.DictReader(fIn, delimiter='\t', quoting=csv.QUOTE_NONE)
    for row in reader:
        if row['qid1'] in dev_sentences and row['qid2'] in dev_sentences:
            dev_duplicates.append([row['qid1'], row['qid2']])

In [ ]:
# The ParaphraseMiningEvaluator computes the cosine similarity between all sentences and
# extracts a list with the pairs that have the highest similarity. Given the duplicate
# information in dev_duplicates, it then computes and F1 score how well our duplicate mining worked
paraphrase_mining_evaluator = evaluation.ParaphraseMiningEvaluator(dev_sentences, 
                                                                   dev_duplicates, name='dev')
evaluators.append(paraphrase_mining_evaluator)

In [ ]:
###### Duplicate Questions Information Retrieval ######
# Given a question and a large corpus of thousands questions, find the most relevant (i.e. duplicate) question
# in that corpus.

# For faster processing, we limit the development corpus to only 10,000 sentences.
# max_corpus_size = 100000
max_corpus_size = 1000

ir_queries = {}             #Our queries (qid => question)
ir_needed_qids = set()      #QIDs we need in the corpus
ir_corpus = {}              #Our corpus (qid => question)
ir_relevant_docs = {}       #Mapping of relevant documents for a given query (qid => set([relevant_question_ids])

with open(os.path.join(dataset_path, 'information-retrieval/dev-queries.tsv'), encoding='utf8') as fIn:
    next(fIn) #Skip header
    for line in fIn:
        qid, query, duplicate_ids = line.strip().split('\t')
        duplicate_ids = duplicate_ids.split(',')
        ir_queries[qid] = query
        ir_relevant_docs[qid] = set(duplicate_ids)

        for qid in duplicate_ids:
            ir_needed_qids.add(qid)

In [ ]:
# First get all needed relevant documents (i.e., we must ensure, that the relevant questions are actually in the corpus
distraction_questions = {}
with open(os.path.join(dataset_path, 'information-retrieval/corpus.tsv'), encoding='utf8') as fIn:
    next(fIn) #Skip header
    for line in fIn:
        qid, question = line.strip().split('\t')

        if qid in ir_needed_qids:
            ir_corpus[qid] = question
        else:
            distraction_questions[qid] = question

# Now, also add some irrelevant questions to fill our corpus
other_qid_list = list(distraction_questions.keys())
random.shuffle(other_qid_list)

for qid in other_qid_list[0:max(0, max_corpus_size-len(ir_corpus))]:
    ir_corpus[qid] = distraction_questions[qid]

In [ ]:
#Given queries, a corpus and a mapping with relevant documents, the InformationRetrievalEvaluator computes different IR
# metrices. For our use case MRR@k and Accuracy@k are relevant.
ir_evaluator = evaluation.InformationRetrievalEvaluator(ir_queries, ir_corpus, ir_relevant_docs)

evaluators.append(ir_evaluator)

In [ ]:
# Create a SequentialEvaluator. This SequentialEvaluator runs all three evaluators in a sequential order.
# We optimize the model with respect to the score from the last evaluator (scores[-1])
seq_evaluator = evaluation.SequentialEvaluator(evaluators, 
                                               main_score_function=lambda scores: scores[-1])


logger.info("Evaluate model without training")
with torch.no_grad():
    seq_evaluator(model, epoch=0, steps=0, output_path=model_save_path)

In [ ]:
# Train the model
model.fit(train_objectives=[(train_dataloader, train_loss)],
          evaluator=seq_evaluator,
          epochs=num_epochs,
#           warmup_steps=1000,
          warmup_steps=500,
          output_path=model_save_path
         )